<a href="https://colab.research.google.com/github/jbenasuli/final_project/blob/database/dev/database/Amazon_Vine-PySpark-ETLs/PRELIM_ETL_Office_Products.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.1.1'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [15.9 kB]
Ign:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:10 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:13 https://

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-05-03 03:24:55--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.68MB/s    in 0.6s    

2021-05-03 03:24:56 (1.68 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
import datetime
from pyspark.sql.functions import to_date
from pyspark.sql.functions import col
from pyspark.sql.types import StructField, StringType, IntegerType, StructType, BooleanType, DateType
from pyspark import SparkFiles

### Load Amazon Data into Spark DataFrame

Note: Enter URL for Desired Dataset

In [5]:
#1 load product segment
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   43081963|R18RVCKGH1SSI9|B001BM2MAC|     307809868|Scotch Cushion Wr...| Office Products|          5|            0|          0|   N|                Y|          Five Stars|      Great product.| 2015-08-31|
|         US|   10951564|R3L4L6LW1PUOFY|B00DZYEXPQ|      75004341|Dust-Off Compress...| Office Products|          5|    

### Create DataFrame - Perform Preliminary Cleaning

In [6]:
#1 Check the schema, print row & column count
df.printSchema()
print((df.count(), len(df.columns)))

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)

(2642434, 15)


In [7]:
#2 Drop the round 1 columns
columns_to_drop = ['marketplace', 'product_parent', 'vine', 'review_headline', 'review_body', 'review_date']
df_dropped = df.drop(*columns_to_drop)
df_dropped.show()

+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+-----------------+
|customer_id|     review_id|product_id|       product_title|product_category|star_rating|helpful_votes|total_votes|verified_purchase|
+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+-----------------+
|   43081963|R18RVCKGH1SSI9|B001BM2MAC|Scotch Cushion Wr...| Office Products|          5|            0|          0|                Y|
|   10951564|R3L4L6LW1PUOFY|B00DZYEXPQ|Dust-Off Compress...| Office Products|          5|            0|          1|                Y|
|   21143145|R2J8AWXWTDX2TF|B00RTMUHDW|Amram Tagger Stan...| Office Products|          5|            0|          0|                Y|
|   52782374|R1PR37BR7G3M6A|B00D7H8XB6|AmazonBasics 12-S...| Office Products|          1|            2|          3|                Y|
|   24045652|R3BDDDZMZBZDPU|B001XCWP34|Derwent Colored P...| O

In [8]:
#3 Filter Step 1:
df_filtered = df_dropped.filter(df_dropped.verified_purchase == 'Y')
df_filtered.show
print((df_filtered.count(), len(df_filtered.columns)))

(2257866, 9)


In [9]:
#4 drop filtered verified_purchase column
columns_to_drop = ['verified_purchase']
df_dropped_2 = df_filtered.drop(*columns_to_drop)
df_dropped_2.show()

+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+
|customer_id|     review_id|product_id|       product_title|product_category|star_rating|helpful_votes|total_votes|
+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+
|   43081963|R18RVCKGH1SSI9|B001BM2MAC|Scotch Cushion Wr...| Office Products|          5|            0|          0|
|   10951564|R3L4L6LW1PUOFY|B00DZYEXPQ|Dust-Off Compress...| Office Products|          5|            0|          1|
|   21143145|R2J8AWXWTDX2TF|B00RTMUHDW|Amram Tagger Stan...| Office Products|          5|            0|          0|
|   52782374|R1PR37BR7G3M6A|B00D7H8XB6|AmazonBasics 12-S...| Office Products|          1|            2|          3|
|   24045652|R3BDDDZMZBZDPU|B001XCWP34|Derwent Colored P...| Office Products|          4|            0|          0|
|   21751234| R8T6MO75ND212|B004J2NBCO|Quartet Magnetic ...| Office Prod

## Create Analysis-Specific DFs/Tables
### Perform Analysis-Specific Transforms

### Segmentation Analysis DF

In [10]:
# 1 Create Segmentation DF by Droppig Addtional Columns
segmentation_cols_drop = ['review_id', 'product_id', 'product_title', 'star_rating', 'helpful_votes', 'total_votes']
segmentation_dropped_df = df_dropped_2.drop(*segmentation_cols_drop)
segmentation_dropped_df.show()

+-----------+----------------+
|customer_id|product_category|
+-----------+----------------+
|   43081963| Office Products|
|   10951564| Office Products|
|   21143145| Office Products|
|   52782374| Office Products|
|   24045652| Office Products|
|   21751234| Office Products|
|    9967215| Office Products|
|   11234247| Office Products|
|   12731488| Office Products|
|    1541556| Office Products|
|   10579374| Office Products|
|   43622307| Office Products|
|   50773856| Office Products|
|   10165223| Office Products|
|   14744748| Office Products|
|   37536631| Office Products|
|     126687| Office Products|
|   31130095| Office Products|
|   14300387| Office Products|
|   22083223| Office Products|
+-----------+----------------+
only showing top 20 rows



Note: must change Category Label name in withColumnRenamed('count(product_category)', 'name')

In [11]:
#2 Segmentation GroupBy
#2a GroupBy customer_id
#2b Count product_category and rename count columns as Segment Name
segment_df = segmentation_dropped_df.groupby("customer_id")\
.agg({'product_category':'count'}).withColumnRenamed('count(product_category)', 'office_products')
#2c Check results
segment_df.show()

+-----------+---------------+
|customer_id|office_products|
+-----------+---------------+
|   43622307|              1|
|   33972406|              1|
|     108460|              1|
|   43515569|              1|
|     132406|              1|
|   42560427|              1|
|    1673863|              2|
|   14552054|              1|
|   45632184|              1|
|   14703850|              2|
|   35768925|              3|
|   45640757|             28|
|   33194403|              1|
|   39383689|              1|
|   10645124|              1|
|   52512151|             10|
|   14646714|              1|
|   38659596|              1|
|   15634680|              1|
|   44178035|              1|
+-----------+---------------+
only showing top 20 rows



In [12]:
#3 Check segment_df Schema and Row Count
segment_df.printSchema()
print(segment_df.count())

root
 |-- customer_id: integer (nullable = true)
 |-- office_products: long (nullable = false)

1557076


Note: Column Name in df.sort('name' ...) must align with Column name from step 2

In [13]:
#4 Filter for Top n Results
#4a Declare number of rows to filter by (100,000)
row_count = 100000
#4a Sort by Segment Desc and limit to row_count
filtered_segment_df = segment_df.sort('office_products', ascending=False).limit(row_count)
#4b Check Results
filtered_segment_df.show()
print(filtered_segment_df.count())

+-----------+---------------+
|customer_id|office_products|
+-----------+---------------+
|   46054413|            121|
|    5044927|            117|
|   16016941|            104|
|   50749068|            101|
|   44115969|             99|
|   44062423|             99|
|   39061233|             97|
|   51148531|             84|
|   35321242|             79|
|   50820654|             76|
|   11474525|             72|
|   13521389|             69|
|     544143|             69|
|   38398088|             69|
|   52443411|             69|
|   51126130|             68|
|   49452274|             66|
|   41309704|             66|
|   33928846|             66|
|   11451031|             66|
+-----------+---------------+
only showing top 20 rows

100000


### Segmentation ETL Complete - Add Database Export Code

In [14]:
# Configure settings for RDS
mode = "append"
jdbc_url='jdbc:postgresql://<connection_srtring>'
config = {"user":'<username>', 
          "password": '<password>', 
          "driver":"org.postgresql.Driver"}

Note: table name in table='name_segment' must align with table name in Postgres

In [15]:
# Write segment table to Postgres/RDS
# xx mins
filtered_segment_df.write.jdbc(url=jdbc_url, table='office_products_segment', mode=mode, properties=config)

## Apriori Analysis DF
### Enter Apriori Transfomations Below - Use df_dropped_2 as Start Point

In [17]:
# 1 Create Apriori DF by First Dropping Addtional Columns
apriori_cols_drop = ['product_category', 'product_title', 'star_rating', 'helpful_votes', 'total_votes']
apriori_dropped_df = df_dropped_2.drop(*apriori_cols_drop)
apriori_dropped_df.show()

+-----------+--------------+----------+
|customer_id|     review_id|product_id|
+-----------+--------------+----------+
|   43081963|R18RVCKGH1SSI9|B001BM2MAC|
|   10951564|R3L4L6LW1PUOFY|B00DZYEXPQ|
|   21143145|R2J8AWXWTDX2TF|B00RTMUHDW|
|   52782374|R1PR37BR7G3M6A|B00D7H8XB6|
|   24045652|R3BDDDZMZBZDPU|B001XCWP34|
|   21751234| R8T6MO75ND212|B004J2NBCO|
|    9967215|R1V2HYL6OI9V39|B003AHIK7U|
|   11234247|R3BLQBKUNXGFS4|B006TKH2RO|
|   12731488|R17MOWJCAR9Y8Q|B00W61M9K0|
|    1541556|R2797NKTEX5THN|B0018RHWGE|
|   10579374|R2RUTMJS50E4WF|B00IA8H2OG|
|   43622307|R1FZYL1HX7O4RM|B007TOIA66|
|   50773856|R22ENT0KU5CHPX|B00EANURCE|
|   10165223|R2E5M7FWJC9JLE|B00T3Q0EOO|
|   14744748|R31BNW3O2Q5J3E|B00WN0N818|
|   37536631|R1JMQF1OOMNF49|B001GAOTSW|
|     126687| RJH6WQDOA6CZP|B003BNY7PW|
|   31130095|R2EVL0Z5JVV86X|B00092RJX0|
|   14300387| R8NWAB2MG94Z6|B000050FZP|
|   22083223|R30C4P6E397CB7|B00EUI162O|
+-----------+--------------+----------+
only showing top 20 rows



### Prelim Apriori ETL Complete - Add Database Export Code

In [18]:
# Configure settings for RDS
mode = "append"
jdbc_url='jdbc:postgresql://<connection_srtring>'
config = {"user":'<username>', 
          "password": '<password>', 
          "driver":"org.postgresql.Driver"}

Note: table name in table='segment_apriori' must align with table name in Postgres

In [19]:
# Write segment_apriori table to RDS
# 15 mins 40 secs
apriori_dropped_df.write.jdbc(url=jdbc_url, table='office_products_apriori', mode=mode, properties=config)

## Amazon Reviews S3 -> Postgres/RDS ETL Complete



### Run Queries in Postgres to Confirm Segment ETL
Check Row Count of Segment Table - SELECT COUNT(*) FROM office_products_segment;

Check 10 Rows of Segment Table - SELECT * FROM office_products_segment LIMIT(10);


In [16]:
# Upon Confirmation of Above Checks Run This Cell
print('Segment ETL Successful')

Segment ETL Successful


### Run Queries in Postgres to Confirm Apriori ETL
Check Row Count of Apriori Table - SELECT COUNT(*) FROM office_products_apriori;

Check 10 Rows of Apriori Table - SELECT * FROM office_products_apriori LIMIT(10);

In [20]:
# Upon Confirmation of Above Checks Run This Cell
print('Apriori ETL Successful')

Apriori ETL Successful
